<div align="center">
  <img width="600px" src="https://www.collinsdictionary.com/images/full/baseball_557405302_1000.jpg">
</div>

# Baseball Salary Predict

**Authors**:
- Cristhian Castillo
- Kevin Zarama
- Aura Rivera
- Jesus Gomez

In this notebook, we are going to work on building models to predict the salary of baseball players.

**Notebook Objetive**: Obtain a reliable model to predict the salary of baseball players.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Packages-and-Functions" data-toc-modified-id="Packages-and-Functions-1">Packages and Functions</a></span><ul class="toc-item"><li><span><a href="#Packages" data-toc-modified-id="Packages-1.1">Packages</a></span></li><li><span><a href="#Custom-Functions" data-toc-modified-id="Custom-Functions-1.2">Custom Functions</a></span></li></ul></li><li><span><a href="#Load-datasets" data-toc-modified-id="Load-datasets-2">Load datasets</a></span></li><li><span><a href="#Data-Preparation" data-toc-modified-id="Data-Preparation-3">Data Preparation</a></span><ul class="toc-item"><li><span><a href="#Remove-Variables" data-toc-modified-id="Remove-Variables-3.1">Remove Variables</a></span></li><li><span><a href="#Separation-of-variables" data-toc-modified-id="Separation-of-variables-3.2">Separation of variables</a></span></li><li><span><a href="#One-Hot-Encoder" data-toc-modified-id="One-Hot-Encoder-3.3">One Hot Encoder</a></span></li></ul></li><li><span><a href="#Baseline" data-toc-modified-id="Baseline-4">Baseline</a></span></li><li><span><a href="#Models" data-toc-modified-id="Models-5">Models</a></span><ul class="toc-item"><li><span><a href="#Load-Hyperparameters" data-toc-modified-id="Load-Hyperparameters-5.1">Load Hyperparameters</a></span></li><li><span><a href="#Base-models" data-toc-modified-id="Base-models-5.2">Base models</a></span></li><li><span><a href="#Models-Hyperparameter-Optimization" data-toc-modified-id="Models-Hyperparameter-Optimization-5.3">Models Hyperparameter Optimization</a></span></li><li><span><a href="#Models-With-Data-Scaling" data-toc-modified-id="Models-With-Data-Scaling-5.4">Models With Data Scaling</a></span></li><li><span><a href="#Models-With-Data-Scaling-Hyperparameter-Optimization" data-toc-modified-id="Models-With-Data-Scaling-Hyperparameter-Optimization-5.5">Models With Data Scaling Hyperparameter Optimization</a></span></li><li><span><a href="#Models-with-numeric-variables" data-toc-modified-id="Models-with-numeric-variables-5.6">Models with numeric variables</a></span></li><li><span><a href="#Models-with-numeric-variables-hyperparameter-optimization" data-toc-modified-id="Models-with-numeric-variables-hyperparameter-optimization-5.7">Models with numeric variables hyperparameter optimization</a></span></li><li><span><a href="#Models-with-numeric-variables-and-data-scaling" data-toc-modified-id="Models-with-numeric-variables-and-data-scaling-5.8">Models with numeric variables and data scaling</a></span></li><li><span><a href="#Models-with-numeric-variables-and-data-scaling-hyperparameter-optimization" data-toc-modified-id="Models-with-numeric-variables-and-data-scaling-hyperparameter-optimization-5.9">Models with numeric variables and data scaling hyperparameter optimization</a></span></li><li><span><a href="#Models-without-outliers" data-toc-modified-id="Models-without-outliers-5.10">Models without outliers</a></span></li><li><span><a href="#Models-without-outliers-hyperparameter-optimization" data-toc-modified-id="Models-without-outliers-hyperparameter-optimization-5.11">Models without outliers hyperparameter optimization</a></span></li><li><span><a href="#Models-without-outliers-data-scaling" data-toc-modified-id="Models-without-outliers-data-scaling-5.12">Models without outliers data scaling</a></span></li><li><span><a href="#Models-without-outliers-data-scaling-hyperparameter-optimization" data-toc-modified-id="Models-without-outliers-data-scaling-hyperparameter-optimization-5.13">Models without outliers data scaling hyperparameter optimization</a></span></li><li><span><a href="#Models-whith-numeric-variables,-without-outliers" data-toc-modified-id="Models-whith-numeric-variables,-without-outliers-5.14">Models whith numeric variables, without outliers</a></span></li><li><span><a href="#Models-with-numeric-variables,-without-outliers-hyperparameter-optimization" data-toc-modified-id="Models-with-numeric-variables,-without-outliers-hyperparameter-optimization-5.15">Models with numeric variables, without outliers hyperparameter optimization</a></span></li><li><span><a href="#Models-with-numeric-variables,-without-outliers-data-scaling" data-toc-modified-id="Models-with-numeric-variables,-without-outliers-data-scaling-5.16">Models with numeric variables, without outliers data scaling</a></span></li><li><span><a href="#Models-without-outliers-data-scaling-hyperparameter-optimization" data-toc-modified-id="Models-without-outliers-data-scaling-hyperparameter-optimization-5.17">Models without outliers data scaling hyperparameter optimization</a></span></li><li><span><a href="#VIF-Analysis" data-toc-modified-id="VIF-Analysis-5.18">VIF Analysis</a></span></li><li><span><a href="#Models-PCA" data-toc-modified-id="Models-PCA-5.19">Models PCA</a></span></li><li><span><a href="#Models-PCA-Hyperparameter-Optimization" data-toc-modified-id="Models-PCA-Hyperparameter-Optimization-5.20">Models PCA Hyperparameter Optimization</a></span></li></ul></li><li><span><a href="#Predict-Data" data-toc-modified-id="Predict-Data-6">Predict Data</a></span></li><li><span><a href="#VIF" data-toc-modified-id="VIF-7">VIF</a></span></li><li><span><a href="#PCA" data-toc-modified-id="PCA-8">PCA</a></span></li><li><span><a href="#PCA-in-Correlacted-Variables" data-toc-modified-id="PCA-in-Correlacted-Variables-9">PCA in Correlacted Variables</a></span></li><li><span><a href="#Feature-Selecting" data-toc-modified-id="Feature-Selecting-10">Feature Selecting</a></span><ul class="toc-item"><li><span><a href="#KBest-with-f_classif" data-toc-modified-id="KBest-with-f_classif-10.1">KBest with f_classif</a></span></li><li><span><a href="#Data-Scalling" data-toc-modified-id="Data-Scalling-10.2">Data Scalling</a></span></li></ul></li><li><span><a href="#Base-Model" data-toc-modified-id="Base-Model-11">Base Model</a></span></li></ul></div>

## Packages and Functions

### Packages

In [1]:
import pandas as pd
import numpy as np
import math

import matplotlib.pyplot as plt


from sklearn.preprocessing import StandardScaler

# Models
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

# Pipelines
from sklearn.pipeline import Pipeline

# Metrics
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error

from sklearn.feature_selection import f_classif

# 

from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import train_test_split, cross_val_predict, HalvingGridSearchCV
from xgboost import plot_importance
from sklearn.feature_selection import SelectFromModel

In [2]:
import warnings
warnings.filterwarnings('ignore')

### Custom Functions 

In [3]:
import sys
sys.path.append("utils/")
import importlib

In [4]:
from pipelines import base_models_pipelines, models_data_scaling_pipelines, models_pca_pipelines
from pipelines import train_models

# Hyperparameters
from hparams_search import generate_params, hyperparameter_optimization, generate_pca_params

from custom_data_preprocessing import get_cols_by_type, one_hot_encoder

from custom_metrics import get_linear_metrics
from custom_statistics import vif_analysis, get_outliers_info

In [5]:
importlib.reload(sys.modules["pipelines"])
importlib.reload(sys.modules["hparams_search"])
importlib.reload(sys.modules["custom_statistics"])

<module 'custom_statistics' from '/workspace/Master of Data Science/Analytics Fundamentals/Basketball Final Project/utils/custom_statistics.py'>

## Load datasets

In [6]:
df = pd.read_csv("datasets/Baseball_Clean.csv")

In [7]:
df_predict = pd.read_csv("datasets/Baseball_Predict.csv")

## Data Preparation

### Remove Variables

In [8]:
df = df.drop(["Player", "Unnamed: 0"], axis=1)

### Separation of variables

In [9]:
y = df["Salary"]
X = df.drop(["Salary"], axis=1)

### One Hot Encoder

We do One Hot Encoder in the categorical variables to train the models.

In [10]:
cat_cols, num_cols =  get_cols_by_type(X)

In [11]:
X = one_hot_encoder(X, cat_cols, drop_first=True)

## Baseline

We calculate the baseline in order to have a reference point. For the baseline we decided to use the RMSE, this metric is what we will use to compare the models. Starting from the baseline metric, models will be trained to obtain a lower RMSE.

In [12]:
baseline_predict = np.full(len(y), y.mean())
baseline_metrics = get_linear_metrics(y, baseline_predict)
baseline_metrics

{'rmse': 450.26022382434286,
 'r2': 0.0,
 'r2_adjusted': 0.5,
 'mae': 343.6183374633141}

The RMSE of the baseline is around 450. We will seek to reduce this metric in the models that we will train.

## Models

To carry out the training of the models we will follow the following route:

1. We will define the base models, the base models are the models that we chose (12) to train to face the problem, which are:
    
    1. Linear Regression.
    2. Ridge.
    3. Lasso.
    4. ElasticNet.
    5. KNN.
    6. CART.
    7. Random Forest Regressor
    8. SVR.
    9. XGBoost.
    10. LGBM Regressor.
    
 We will train these models with the entire data set, without treatments (only the one hot encode) and we will obtain their metrics.

2. From here we will look for the best hyperparameters and ways of treating the data set that gives us the best results.

### Load Hyperparameters

In [13]:
models_params = generate_params()
pca_params = generate_pca_params()

### Base models

We will train the models that we will take into consideration using all the predictor variables of the dataset

In [14]:
pipelines_base_models = base_models_pipelines()

In [15]:
base_models = train_models(X, y, pipelines=pipelines_base_models, cv=5)

In [16]:
base_models

,model,rmse,regressor
0,LinearRegression,342.139591,(LinearRegression())
1,Ridge,341.721405,(Ridge())
2,Lasso,340.862121,(Lasso())
3,ElasticNet,339.059278,(ElasticNet())
4,KNN,326.886890,(KNeighborsRegressor())
5,CART,400.368252,(DecisionTreeRegressor())
6,RandomForestRegressor,284.201486,(RandomForestRegressor())
7,SVR,444.179645,(SVR())
8,XGBoost,310.077887,"(XGBRegressor(base_score=None, booster=None, c..."
9,LGBMRegressor,296.697898,(LGBMRegressor())


Now we have some base models, which have considerable improvements compared to the baseline. Except for SVR, which has initial results almost the same as the baseline.

### Models Hyperparameter Optimization

We will do a hyperparameter search in order to improve the results of our base models.

In [17]:
base_models_hparams = hyperparameter_optimization(X, y, pipelines=pipelines_base_models, models_params=models_params, save_path="models/hs_base_models/")

LinearRegression Hyperparameter Tuning...
Not Hyperparameter Tuning for LinearRegression
Loading Ridge from pickle file...
RMSE: 341.4025933516834
Ridge best params: {'memory': None, 'steps': [('Ridge', Ridge(alpha=2.0, solver='svd'))], 'verbose': False, 'Ridge': Ridge(alpha=2.0, solver='svd'), 'Ridge__alpha': 2.0, 'Ridge__copy_X': True, 'Ridge__fit_intercept': True, 'Ridge__max_iter': None, 'Ridge__normalize': False, 'Ridge__random_state': None, 'Ridge__solver': 'svd', 'Ridge__tol': 0.001}
Loading Lasso from pickle file...
RMSE: 340.2040215266149
Lasso best params: {'memory': None, 'steps': [('Lasso', Lasso(alpha=2.0))], 'verbose': False, 'Lasso': Lasso(alpha=2.0), 'Lasso__alpha': 2.0, 'Lasso__copy_X': True, 'Lasso__fit_intercept': True, 'Lasso__max_iter': 1000, 'Lasso__normalize': False, 'Lasso__positive': False, 'Lasso__precompute': False, 'Lasso__random_state': None, 'Lasso__selection': 'cyclic', 'Lasso__tol': 0.0001, 'Lasso__warm_start': False}
Loading ElasticNet from pickle file.

In [18]:
base_models_hparams

{'LinearRegression': {'model': Pipeline(steps=[('LinearRegression', LinearRegression())]),
  'rmse': 342.1395913172409,
  'params': None},
 'Ridge': {'model': Pipeline(steps=[('Ridge', Ridge(alpha=2.0, solver='svd'))]),
  'rmse': 341.4025933516834,
  'params': {'memory': None,
   'steps': [('Ridge', Ridge(alpha=2.0, solver='svd'))],
   'verbose': False,
   'Ridge': Ridge(alpha=2.0, solver='svd'),
   'Ridge__alpha': 2.0,
   'Ridge__copy_X': True,
   'Ridge__fit_intercept': True,
   'Ridge__max_iter': None,
   'Ridge__normalize': False,
   'Ridge__random_state': None,
   'Ridge__solver': 'svd',
   'Ridge__tol': 0.001}},
 'Lasso': {'model': Pipeline(steps=[('Lasso', Lasso(alpha=2.0))]),
  'rmse': 340.2040215266149,
  'params': {'memory': None,
   'steps': [('Lasso', Lasso(alpha=2.0))],
   'verbose': False,
   'Lasso': Lasso(alpha=2.0),
   'Lasso__alpha': 2.0,
   'Lasso__copy_X': True,
   'Lasso__fit_intercept': True,
   'Lasso__max_iter': 1000,
   'Lasso__normalize': False,
   'Lasso__pos

At the moment our best model is LGBMRegressor, after hyperparameter optimization with an RMSE of 287.765

### Models With Data Scaling 

In [19]:
pipelines_models_data_scaling = models_data_scaling_pipelines()

In [20]:
models_data_scaling = train_models(X, y, pipelines=pipelines_models_data_scaling, cv=5)

In [21]:
models_data_scaling

,model,rmse,regressor
0,LinearRegression,342.139591,"(StandardScaler(), LinearRegression())"
1,Ridge,339.683083,"(StandardScaler(), Ridge())"
2,Lasso,340.795470,"(StandardScaler(), Lasso())"
3,ElasticNet,339.524577,"(StandardScaler(), ElasticNet())"
4,KNN,345.033061,"(StandardScaler(), KNeighborsRegressor())"
5,CART,370.019727,"(StandardScaler(), DecisionTreeRegressor())"
6,RandomForestRegressor,284.033237,"(StandardScaler(), RandomForestRegressor())"
7,SVR,448.423651,"(StandardScaler(), SVR())"
8,XGBoost,312.011893,"(StandardScaler(), XGBRegressor(base_score=Non..."
9,LGBMRegressor,297.014718,"(StandardScaler(), LGBMRegressor())"


With data scaling we managed to obtain a RandomForestRegressor model with an RMSE of 275,601, so it would be the best model found so far.

### Models With Data Scaling Hyperparameter Optimization

In [22]:
models_data_scaling_hparams = hyperparameter_optimization(X, y, pipelines=pipelines_models_data_scaling, models_params=models_params, save_path="models/hs_models_data_scaling/")

LinearRegression Hyperparameter Tuning...
Not Hyperparameter Tuning for LinearRegression
Loading Ridge from pickle file...
RMSE: 339.40565549752915
Ridge best params: {'memory': None, 'steps': [('StandardScaler', StandardScaler()), ('Ridge', Ridge(alpha=2.0))], 'verbose': False, 'StandardScaler': StandardScaler(), 'Ridge': Ridge(alpha=2.0), 'StandardScaler__copy': True, 'StandardScaler__with_mean': True, 'StandardScaler__with_std': True, 'Ridge__alpha': 2.0, 'Ridge__copy_X': True, 'Ridge__fit_intercept': True, 'Ridge__max_iter': None, 'Ridge__normalize': False, 'Ridge__random_state': None, 'Ridge__solver': 'auto', 'Ridge__tol': 0.001}
Loading Lasso from pickle file...
RMSE: 340.33462091683646
Lasso best params: {'memory': None, 'steps': [('StandardScaler', StandardScaler()), ('Lasso', Lasso(alpha=1.7000000000000002, max_iter=100))], 'verbose': False, 'StandardScaler': StandardScaler(), 'Lasso': Lasso(alpha=1.7000000000000002, max_iter=100), 'StandardScaler__copy': True, 'StandardScaler

RMSE: 287.09697818431493
LGBMRegressor best params: {'memory': None, 'steps': [('StandardScaler', StandardScaler()), ('LGBMRegressor', LGBMRegressor(max_depth=2))], 'verbose': False, 'StandardScaler': StandardScaler(), 'LGBMRegressor': LGBMRegressor(max_depth=2), 'StandardScaler__copy': True, 'StandardScaler__with_mean': True, 'StandardScaler__with_std': True, 'LGBMRegressor__boosting_type': 'gbdt', 'LGBMRegressor__class_weight': None, 'LGBMRegressor__colsample_bytree': 1.0, 'LGBMRegressor__importance_type': 'split', 'LGBMRegressor__learning_rate': 0.1, 'LGBMRegressor__max_depth': 2, 'LGBMRegressor__min_child_samples': 20, 'LGBMRegressor__min_child_weight': 0.001, 'LGBMRegressor__min_split_gain': 0.0, 'LGBMRegressor__n_estimators': 100, 'LGBMRegressor__n_jobs': -1, 'LGBMRegressor__num_leaves': 31, 'LGBMRegressor__objective': None, 'LGBMRegressor__random_state': None, 'LGBMRegressor__reg_alpha': 0.0, 'LGBMRegressor__reg_lambda': 0.0, 'LGBMRegressor__silent': True, 'LGBMRegressor__subsam

### Models with numeric variables

In [23]:
numeric_models = train_models(X[num_cols], y, pipelines=pipelines_base_models, cv=5)

In [24]:
numeric_models

,model,rmse,regressor
0,LinearRegression,341.460887,(LinearRegression())
1,Ridge,341.457007,(Ridge())
2,Lasso,341.108329,(Lasso())
3,ElasticNet,340.839900,(ElasticNet())
4,KNN,326.886890,(KNeighborsRegressor())
5,CART,377.875028,(DecisionTreeRegressor())
6,RandomForestRegressor,282.037047,(RandomForestRegressor())
7,SVR,444.069592,(SVR())
8,XGBoost,304.707971,"(XGBRegressor(base_score=None, booster=None, c..."
9,LGBMRegressor,297.392872,(LGBMRegressor())


### Models with numeric variables hyperparameter optimization

In [25]:
numeric_models_hparams = hyperparameter_optimization(X[num_cols], y, pipelines=pipelines_base_models, models_params=models_params, save_path="models/hs_numeric_models/")

LinearRegression Hyperparameter Tuning...
Not Hyperparameter Tuning for LinearRegression
Loading Ridge from pickle file...
RMSE: 341.79726132868456
Ridge best params: {'memory': None, 'steps': [('Ridge', Ridge(alpha=2.0, solver='lsqr'))], 'verbose': False, 'Ridge': Ridge(alpha=2.0, solver='lsqr'), 'Ridge__alpha': 2.0, 'Ridge__copy_X': True, 'Ridge__fit_intercept': True, 'Ridge__max_iter': None, 'Ridge__normalize': False, 'Ridge__random_state': None, 'Ridge__solver': 'lsqr', 'Ridge__tol': 0.001}
Loading Lasso from pickle file...
RMSE: 340.9887520432366
Lasso best params: {'memory': None, 'steps': [('Lasso', Lasso(alpha=2.0))], 'verbose': False, 'Lasso': Lasso(alpha=2.0), 'Lasso__alpha': 2.0, 'Lasso__copy_X': True, 'Lasso__fit_intercept': True, 'Lasso__max_iter': 1000, 'Lasso__normalize': False, 'Lasso__positive': False, 'Lasso__precompute': False, 'Lasso__random_state': None, 'Lasso__selection': 'cyclic', 'Lasso__tol': 0.0001, 'Lasso__warm_start': False}
Loading ElasticNet from pickle f

In [26]:
numeric_models_hparams

{'LinearRegression': {'model': Pipeline(steps=[('LinearRegression', LinearRegression())]),
  'rmse': 341.4608874867939,
  'params': None},
 'Ridge': {'model': Pipeline(steps=[('Ridge', Ridge(alpha=2.0, solver='lsqr'))]),
  'rmse': 341.79726132868456,
  'params': {'memory': None,
   'steps': [('Ridge', Ridge(alpha=2.0, solver='lsqr'))],
   'verbose': False,
   'Ridge': Ridge(alpha=2.0, solver='lsqr'),
   'Ridge__alpha': 2.0,
   'Ridge__copy_X': True,
   'Ridge__fit_intercept': True,
   'Ridge__max_iter': None,
   'Ridge__normalize': False,
   'Ridge__random_state': None,
   'Ridge__solver': 'lsqr',
   'Ridge__tol': 0.001}},
 'Lasso': {'model': Pipeline(steps=[('Lasso', Lasso(alpha=2.0))]),
  'rmse': 340.9887520432366,
  'params': {'memory': None,
   'steps': [('Lasso', Lasso(alpha=2.0))],
   'verbose': False,
   'Lasso': Lasso(alpha=2.0),
   'Lasso__alpha': 2.0,
   'Lasso__copy_X': True,
   'Lasso__fit_intercept': True,
   'Lasso__max_iter': 1000,
   'Lasso__normalize': False,
   'Lasso

### Models with numeric variables and data scaling

In [27]:
numeric_models_scaling = train_models(X[num_cols], y, pipelines=pipelines_models_data_scaling, cv=5)

In [28]:
numeric_models_scaling

,model,rmse,regressor
0,LinearRegression,341.460887,"(StandardScaler(), LinearRegression())"
1,Ridge,338.978878,"(StandardScaler(), Ridge())"
2,Lasso,340.074760,"(StandardScaler(), Lasso())"
3,ElasticNet,342.430297,"(StandardScaler(), ElasticNet())"
4,KNN,317.913695,"(StandardScaler(), KNeighborsRegressor())"
5,CART,364.739943,"(StandardScaler(), DecisionTreeRegressor())"
6,RandomForestRegressor,281.303066,"(StandardScaler(), RandomForestRegressor())"
7,SVR,446.670402,"(StandardScaler(), SVR())"
8,XGBoost,306.088513,"(StandardScaler(), XGBRegressor(base_score=Non..."
9,LGBMRegressor,297.145101,"(StandardScaler(), LGBMRegressor())"


### Models with numeric variables and data scaling hyperparameter optimization

In [29]:
numeric_models_scaling_hparams = hyperparameter_optimization(X[num_cols], y, pipelines=pipelines_models_data_scaling, models_params=models_params, save_path="models/hs_numeric_models_scaling/")

LinearRegression Hyperparameter Tuning...
Not Hyperparameter Tuning for LinearRegression
Loading Ridge from pickle file...
RMSE: 338.947898575345
Ridge best params: {'memory': None, 'steps': [('StandardScaler', StandardScaler()), ('Ridge', Ridge(alpha=2.0))], 'verbose': False, 'StandardScaler': StandardScaler(), 'Ridge': Ridge(alpha=2.0), 'StandardScaler__copy': True, 'StandardScaler__with_mean': True, 'StandardScaler__with_std': True, 'Ridge__alpha': 2.0, 'Ridge__copy_X': True, 'Ridge__fit_intercept': True, 'Ridge__max_iter': None, 'Ridge__normalize': False, 'Ridge__random_state': None, 'Ridge__solver': 'auto', 'Ridge__tol': 0.001}
Loading Lasso from pickle file...
RMSE: 340.0599531575751
Lasso best params: {'memory': None, 'steps': [('StandardScaler', StandardScaler()), ('Lasso', Lasso(alpha=2.0))], 'verbose': False, 'StandardScaler': StandardScaler(), 'Lasso': Lasso(alpha=2.0), 'StandardScaler__copy': True, 'StandardScaler__with_mean': True, 'StandardScaler__with_std': True, 'Lasso_

RMSE: 284.5003906277651
LGBMRegressor best params: {'memory': None, 'steps': [('StandardScaler', StandardScaler()), ('LGBMRegressor', LGBMRegressor(max_depth=2))], 'verbose': False, 'StandardScaler': StandardScaler(), 'LGBMRegressor': LGBMRegressor(max_depth=2), 'StandardScaler__copy': True, 'StandardScaler__with_mean': True, 'StandardScaler__with_std': True, 'LGBMRegressor__boosting_type': 'gbdt', 'LGBMRegressor__class_weight': None, 'LGBMRegressor__colsample_bytree': 1.0, 'LGBMRegressor__importance_type': 'split', 'LGBMRegressor__learning_rate': 0.1, 'LGBMRegressor__max_depth': 2, 'LGBMRegressor__min_child_samples': 20, 'LGBMRegressor__min_child_weight': 0.001, 'LGBMRegressor__min_split_gain': 0.0, 'LGBMRegressor__n_estimators': 100, 'LGBMRegressor__n_jobs': -1, 'LGBMRegressor__num_leaves': 31, 'LGBMRegressor__objective': None, 'LGBMRegressor__random_state': None, 'LGBMRegressor__reg_alpha': 0.0, 'LGBMRegressor__reg_lambda': 0.0, 'LGBMRegressor__silent': True, 'LGBMRegressor__subsamp

In [30]:
numeric_models_scaling_hparams

{'LinearRegression': {'model': Pipeline(steps=[('StandardScaler', StandardScaler()),
                  ('LinearRegression', LinearRegression())]),
  'rmse': 341.4608874867931,
  'params': None},
 'Ridge': {'model': Pipeline(steps=[('StandardScaler', StandardScaler()),
                  ('Ridge', Ridge(alpha=2.0))]),
  'rmse': 338.947898575345,
  'params': {'memory': None,
   'steps': [('StandardScaler', StandardScaler()),
    ('Ridge', Ridge(alpha=2.0))],
   'verbose': False,
   'StandardScaler': StandardScaler(),
   'Ridge': Ridge(alpha=2.0),
   'StandardScaler__copy': True,
   'StandardScaler__with_mean': True,
   'StandardScaler__with_std': True,
   'Ridge__alpha': 2.0,
   'Ridge__copy_X': True,
   'Ridge__fit_intercept': True,
   'Ridge__max_iter': None,
   'Ridge__normalize': False,
   'Ridge__random_state': None,
   'Ridge__solver': 'auto',
   'Ridge__tol': 0.001}},
 'Lasso': {'model': Pipeline(steps=[('StandardScaler', StandardScaler()),
                  ('Lasso', Lasso(alpha=2

So far we have not found a better model than the random forest with an RMSE of 275

### Models without outliers

In the EDA analysis we discover that there are values that are far from the mean 4 standard desviation, but these values correspond to real data. However, as it is possible that these outliers are affecting our model, we decided to eliminate them and review the results.

In [31]:
X_without_outliers = X.drop(X.index[[73, 100, 163, 236, 249]])

In [32]:
y_without_outliers = y.drop(y.index[[73, 100, 163, 236, 249]])

In [33]:
models_outliers = train_models(X_without_outliers, y_without_outliers, pipelines=pipelines_base_models, cv=5)

In [34]:
models_outliers

,model,rmse,regressor
0,LinearRegression,343.623892,(LinearRegression())
1,Ridge,343.227504,(Ridge())
2,Lasso,342.344612,(Lasso())
3,ElasticNet,340.899372,(ElasticNet())
4,KNN,332.549686,(KNeighborsRegressor())
5,CART,364.097515,(DecisionTreeRegressor())
6,RandomForestRegressor,284.823268,(RandomForestRegressor())
7,SVR,440.114271,(SVR())
8,XGBoost,308.374095,"(XGBRegressor(base_score=None, booster=None, c..."
9,LGBMRegressor,290.334449,(LGBMRegressor())


Removing the outliers we managed to find a 278.408750, but it's still worse than the 275 from the random forest we found earlier.

We will try with 3.6 std

In [35]:
df_outliers = pd.DataFrame(X)
df_outliers["Salary"] = y
get_outliers_info(df_outliers, nstd=3.6)

{'CAtBat': {'upper_bound': 10889.242272301068,
  'nupper_outliers': 1,
  'upper_outliers': 188    14053
  Name: CAtBat, dtype: int64},
 'CHits': {'upper_bound': 3055.7050292173753,
  'nupper_outliers': 1,
  'upper_outliers': 188    4256
  Name: CHits, dtype: int64},
 'CHmRun': {'upper_bound': 365.1508370242302,
  'nupper_outliers': 2,
  'upper_outliers': 91     384
  200    548
  Name: CHmRun, dtype: int64},
 'CRuns': {'upper_bound': 1553.5353864637275,
  'nupper_outliers': 1,
  'upper_outliers': 188    2165
  Name: CRuns, dtype: int64},
 'CRBI': {'upper_bound': 1494.5418564084016,
  'nupper_outliers': 1,
  'upper_outliers': 200    1659
  Name: CRBI, dtype: int64},
 'CWalks': {'upper_bound': 1210.867284376398,
  'nupper_outliers': 3,
  'upper_outliers': 54     1380
  188    1566
  200    1342
  Name: CWalks, dtype: int64},
 'PutOuts': {'upper_bound': 1298.4754983965108,
  'nupper_outliers': 3,
  'upper_outliers': 61     1377
  221    1320
  261    1314
  Name: PutOuts, dtype: int64},
 

In [36]:
df_without_outliers = df_outliers.drop([54,  61,  76,  91, 126, 188, 200], axis=0)

In [37]:
y_without_outliers2 = df_without_outliers["Salary"]
X_without_outliers2 = df_without_outliers.drop(["Salary"], axis=1)

In [38]:
models_outliers2 = train_models(X_without_outliers2[num_cols], y_without_outliers2, pipelines=pipelines_base_models, cv=10)

In [39]:
models_outliers2

,model,rmse,regressor
0,LinearRegression,302.310170,(LinearRegression())
1,Ridge,302.303977,(Ridge())
2,Lasso,302.011368,(Lasso())
3,ElasticNet,301.500262,(ElasticNet())
4,KNN,304.233294,(KNeighborsRegressor())
5,CART,363.558953,(DecisionTreeRegressor())
6,RandomForestRegressor,248.857616,(RandomForestRegressor())
7,SVR,404.787186,(SVR())
8,XGBoost,266.049053,"(XGBRegressor(base_score=None, booster=None, c..."
9,LGBMRegressor,259.571121,(LGBMRegressor())


### Models without outliers hyperparameter optimization

In [40]:
models_outliers_hparams = hyperparameter_optimization(X_without_outliers, y_without_outliers, pipelines=pipelines_base_models, models_params=models_params, save_path="models/hs_models_outliers/")

LinearRegression Hyperparameter Tuning...
Not Hyperparameter Tuning for LinearRegression
Loading Ridge from pickle file...
RMSE: 344.45249969868485
Ridge best params: {'memory': None, 'steps': [('Ridge', Ridge(alpha=2.0, solver='lsqr'))], 'verbose': False, 'Ridge': Ridge(alpha=2.0, solver='lsqr'), 'Ridge__alpha': 2.0, 'Ridge__copy_X': True, 'Ridge__fit_intercept': True, 'Ridge__max_iter': None, 'Ridge__normalize': False, 'Ridge__random_state': None, 'Ridge__solver': 'lsqr', 'Ridge__tol': 0.001}
Loading Lasso from pickle file...
RMSE: 341.82410837164997
Lasso best params: {'memory': None, 'steps': [('Lasso', Lasso(alpha=2.0))], 'verbose': False, 'Lasso': Lasso(alpha=2.0), 'Lasso__alpha': 2.0, 'Lasso__copy_X': True, 'Lasso__fit_intercept': True, 'Lasso__max_iter': 1000, 'Lasso__normalize': False, 'Lasso__positive': False, 'Lasso__precompute': False, 'Lasso__random_state': None, 'Lasso__selection': 'cyclic', 'Lasso__tol': 0.0001, 'Lasso__warm_start': False}
Loading ElasticNet from pickle 

In [41]:
models_outliers_hparams

{'LinearRegression': {'model': Pipeline(steps=[('LinearRegression', LinearRegression())]),
  'rmse': 343.62389187800187,
  'params': None},
 'Ridge': {'model': Pipeline(steps=[('Ridge', Ridge(alpha=2.0, solver='lsqr'))]),
  'rmse': 344.45249969868485,
  'params': {'memory': None,
   'steps': [('Ridge', Ridge(alpha=2.0, solver='lsqr'))],
   'verbose': False,
   'Ridge': Ridge(alpha=2.0, solver='lsqr'),
   'Ridge__alpha': 2.0,
   'Ridge__copy_X': True,
   'Ridge__fit_intercept': True,
   'Ridge__max_iter': None,
   'Ridge__normalize': False,
   'Ridge__random_state': None,
   'Ridge__solver': 'lsqr',
   'Ridge__tol': 0.001}},
 'Lasso': {'model': Pipeline(steps=[('Lasso', Lasso(alpha=2.0))]),
  'rmse': 341.82410837164997,
  'params': {'memory': None,
   'steps': [('Lasso', Lasso(alpha=2.0))],
   'verbose': False,
   'Lasso': Lasso(alpha=2.0),
   'Lasso__alpha': 2.0,
   'Lasso__copy_X': True,
   'Lasso__fit_intercept': True,
   'Lasso__max_iter': 1000,
   'Lasso__normalize': False,
   'Las

### Models without outliers data scaling

In [42]:
models_outliers_scaling = train_models(X_without_outliers, y_without_outliers, pipelines=pipelines_models_data_scaling, cv=5)

In [43]:
models_outliers_scaling

,model,rmse,regressor
0,LinearRegression,343.623892,"(StandardScaler(), LinearRegression())"
1,Ridge,339.727590,"(StandardScaler(), Ridge())"
2,Lasso,341.739266,"(StandardScaler(), Lasso())"
3,ElasticNet,340.684940,"(StandardScaler(), ElasticNet())"
4,KNN,346.626316,"(StandardScaler(), KNeighborsRegressor())"
5,CART,363.633004,"(StandardScaler(), DecisionTreeRegressor())"
6,RandomForestRegressor,277.696115,"(StandardScaler(), RandomForestRegressor())"
7,SVR,444.960714,"(StandardScaler(), SVR())"
8,XGBoost,310.645172,"(StandardScaler(), XGBRegressor(base_score=Non..."
9,LGBMRegressor,285.807180,"(StandardScaler(), LGBMRegressor())"


### Models without outliers data scaling hyperparameter optimization

In [44]:
models_outliers_scaling_hparams = hyperparameter_optimization(X_without_outliers, y_without_outliers, pipelines=pipelines_base_models, models_params=models_params, save_path="models/hs_models_outliers_scaling/")

LinearRegression Hyperparameter Tuning...
Not Hyperparameter Tuning for LinearRegression
Loading Ridge from pickle file...
RMSE: 344.45249969868485
Ridge best params: {'memory': None, 'steps': [('Ridge', Ridge(alpha=2.0, solver='lsqr'))], 'verbose': False, 'Ridge': Ridge(alpha=2.0, solver='lsqr'), 'Ridge__alpha': 2.0, 'Ridge__copy_X': True, 'Ridge__fit_intercept': True, 'Ridge__max_iter': None, 'Ridge__normalize': False, 'Ridge__random_state': None, 'Ridge__solver': 'lsqr', 'Ridge__tol': 0.001}
Loading Lasso from pickle file...
RMSE: 341.82410837164997
Lasso best params: {'memory': None, 'steps': [('Lasso', Lasso(alpha=2.0))], 'verbose': False, 'Lasso': Lasso(alpha=2.0), 'Lasso__alpha': 2.0, 'Lasso__copy_X': True, 'Lasso__fit_intercept': True, 'Lasso__max_iter': 1000, 'Lasso__normalize': False, 'Lasso__positive': False, 'Lasso__precompute': False, 'Lasso__random_state': None, 'Lasso__selection': 'cyclic', 'Lasso__tol': 0.0001, 'Lasso__warm_start': False}
Loading ElasticNet from pickle 

In [45]:
models_outliers_scaling_hparams

{'LinearRegression': {'model': Pipeline(steps=[('LinearRegression', LinearRegression())]),
  'rmse': 343.62389187800187,
  'params': None},
 'Ridge': {'model': Pipeline(steps=[('Ridge', Ridge(alpha=2.0, solver='lsqr'))]),
  'rmse': 344.45249969868485,
  'params': {'memory': None,
   'steps': [('Ridge', Ridge(alpha=2.0, solver='lsqr'))],
   'verbose': False,
   'Ridge': Ridge(alpha=2.0, solver='lsqr'),
   'Ridge__alpha': 2.0,
   'Ridge__copy_X': True,
   'Ridge__fit_intercept': True,
   'Ridge__max_iter': None,
   'Ridge__normalize': False,
   'Ridge__random_state': None,
   'Ridge__solver': 'lsqr',
   'Ridge__tol': 0.001}},
 'Lasso': {'model': Pipeline(steps=[('Lasso', Lasso(alpha=2.0))]),
  'rmse': 341.82410837164997,
  'params': {'memory': None,
   'steps': [('Lasso', Lasso(alpha=2.0))],
   'verbose': False,
   'Lasso': Lasso(alpha=2.0),
   'Lasso__alpha': 2.0,
   'Lasso__copy_X': True,
   'Lasso__fit_intercept': True,
   'Lasso__max_iter': 1000,
   'Lasso__normalize': False,
   'Las

### Models whith numeric variables, without outliers 

In [46]:
numeric_models_outliers = train_models(X_without_outliers[num_cols], y_without_outliers, pipelines=pipelines_base_models, cv=5)

In [47]:
numeric_models_outliers

,model,rmse,regressor
0,LinearRegression,343.393141,(LinearRegression())
1,Ridge,343.390416,(Ridge())
2,Lasso,343.000709,(Lasso())
3,ElasticNet,342.818469,(ElasticNet())
4,KNN,332.549686,(KNeighborsRegressor())
5,CART,370.312976,(DecisionTreeRegressor())
6,RandomForestRegressor,281.256815,(RandomForestRegressor())
7,SVR,440.045465,(SVR())
8,XGBoost,305.122793,"(XGBRegressor(base_score=None, booster=None, c..."
9,LGBMRegressor,287.990155,(LGBMRegressor())


### Models with numeric variables, without outliers hyperparameter optimization

In [48]:
numeric_models_outliers_hparams = hyperparameter_optimization(X_without_outliers[num_cols], y_without_outliers, pipelines=pipelines_base_models, models_params=models_params, save_path="models/hs_numeric_models_outliers/")

LinearRegression Hyperparameter Tuning...
Not Hyperparameter Tuning for LinearRegression
Loading Ridge from pickle file...
RMSE: 344.4569717766386
Ridge best params: {'memory': None, 'steps': [('Ridge', Ridge(alpha=2.0, solver='lsqr'))], 'verbose': False, 'Ridge': Ridge(alpha=2.0, solver='lsqr'), 'Ridge__alpha': 2.0, 'Ridge__copy_X': True, 'Ridge__fit_intercept': True, 'Ridge__max_iter': None, 'Ridge__normalize': False, 'Ridge__random_state': None, 'Ridge__solver': 'lsqr', 'Ridge__tol': 0.001}
Loading Lasso from pickle file...
RMSE: 342.9164003231138
Lasso best params: {'memory': None, 'steps': [('Lasso', Lasso(alpha=2.0))], 'verbose': False, 'Lasso': Lasso(alpha=2.0), 'Lasso__alpha': 2.0, 'Lasso__copy_X': True, 'Lasso__fit_intercept': True, 'Lasso__max_iter': 1000, 'Lasso__normalize': False, 'Lasso__positive': False, 'Lasso__precompute': False, 'Lasso__random_state': None, 'Lasso__selection': 'cyclic', 'Lasso__tol': 0.0001, 'Lasso__warm_start': False}
Loading ElasticNet from pickle fi

In [49]:
numeric_models_outliers_hparams

{'LinearRegression': {'model': Pipeline(steps=[('LinearRegression', LinearRegression())]),
  'rmse': 343.3931405442098,
  'params': None},
 'Ridge': {'model': Pipeline(steps=[('Ridge', Ridge(alpha=2.0, solver='lsqr'))]),
  'rmse': 344.4569717766386,
  'params': {'memory': None,
   'steps': [('Ridge', Ridge(alpha=2.0, solver='lsqr'))],
   'verbose': False,
   'Ridge': Ridge(alpha=2.0, solver='lsqr'),
   'Ridge__alpha': 2.0,
   'Ridge__copy_X': True,
   'Ridge__fit_intercept': True,
   'Ridge__max_iter': None,
   'Ridge__normalize': False,
   'Ridge__random_state': None,
   'Ridge__solver': 'lsqr',
   'Ridge__tol': 0.001}},
 'Lasso': {'model': Pipeline(steps=[('Lasso', Lasso(alpha=2.0))]),
  'rmse': 342.9164003231138,
  'params': {'memory': None,
   'steps': [('Lasso', Lasso(alpha=2.0))],
   'verbose': False,
   'Lasso': Lasso(alpha=2.0),
   'Lasso__alpha': 2.0,
   'Lasso__copy_X': True,
   'Lasso__fit_intercept': True,
   'Lasso__max_iter': 1000,
   'Lasso__normalize': False,
   'Lasso_

### Models with numeric variables, without outliers data scaling

In [50]:
numeric_models_outliers_scaling = train_models(X_without_outliers[num_cols], y_without_outliers, pipelines=pipelines_models_data_scaling, cv=5)

In [51]:
numeric_models_outliers_scaling

,model,rmse,regressor
0,LinearRegression,343.393141,"(StandardScaler(), LinearRegression())"
1,Ridge,338.959819,"(StandardScaler(), Ridge())"
2,Lasso,340.777527,"(StandardScaler(), Lasso())"
3,ElasticNet,343.021314,"(StandardScaler(), ElasticNet())"
4,KNN,315.366075,"(StandardScaler(), KNeighborsRegressor())"
5,CART,366.060711,"(StandardScaler(), DecisionTreeRegressor())"
6,RandomForestRegressor,278.658030,"(StandardScaler(), RandomForestRegressor())"
7,SVR,443.424788,"(StandardScaler(), SVR())"
8,XGBoost,307.821424,"(StandardScaler(), XGBRegressor(base_score=Non..."
9,LGBMRegressor,288.790608,"(StandardScaler(), LGBMRegressor())"


### Models without outliers data scaling hyperparameter optimization

In [52]:
numeric_models_outliers_scaling_hparams = hyperparameter_optimization(X_without_outliers[num_cols], y_without_outliers, pipelines = pipelines_models_data_scaling, models_params=models_params, save_path="models/hs_numeric_models_outliers_scaling/")

LinearRegression Hyperparameter Tuning...
Not Hyperparameter Tuning for LinearRegression
Loading Ridge from pickle file...
RMSE: 338.57558034018706
Ridge best params: {'memory': None, 'steps': [('StandardScaler', StandardScaler()), ('Ridge', Ridge(alpha=2.0))], 'verbose': False, 'StandardScaler': StandardScaler(), 'Ridge': Ridge(alpha=2.0), 'StandardScaler__copy': True, 'StandardScaler__with_mean': True, 'StandardScaler__with_std': True, 'Ridge__alpha': 2.0, 'Ridge__copy_X': True, 'Ridge__fit_intercept': True, 'Ridge__max_iter': None, 'Ridge__normalize': False, 'Ridge__random_state': None, 'Ridge__solver': 'auto', 'Ridge__tol': 0.001}
Loading Lasso from pickle file...
RMSE: 339.72906981656564
Lasso best params: {'memory': None, 'steps': [('StandardScaler', StandardScaler()), ('Lasso', Lasso(alpha=2.0, max_iter=10000))], 'verbose': False, 'StandardScaler': StandardScaler(), 'Lasso': Lasso(alpha=2.0, max_iter=10000), 'StandardScaler__copy': True, 'StandardScaler__with_mean': True, 'Stand

RMSE: 275.45884808426484
LGBMRegressor best params: {'memory': None, 'steps': [('StandardScaler', StandardScaler()), ('LGBMRegressor', LGBMRegressor(max_depth=2))], 'verbose': False, 'StandardScaler': StandardScaler(), 'LGBMRegressor': LGBMRegressor(max_depth=2), 'StandardScaler__copy': True, 'StandardScaler__with_mean': True, 'StandardScaler__with_std': True, 'LGBMRegressor__boosting_type': 'gbdt', 'LGBMRegressor__class_weight': None, 'LGBMRegressor__colsample_bytree': 1.0, 'LGBMRegressor__importance_type': 'split', 'LGBMRegressor__learning_rate': 0.1, 'LGBMRegressor__max_depth': 2, 'LGBMRegressor__min_child_samples': 20, 'LGBMRegressor__min_child_weight': 0.001, 'LGBMRegressor__min_split_gain': 0.0, 'LGBMRegressor__n_estimators': 100, 'LGBMRegressor__n_jobs': -1, 'LGBMRegressor__num_leaves': 31, 'LGBMRegressor__objective': None, 'LGBMRegressor__random_state': None, 'LGBMRegressor__reg_alpha': 0.0, 'LGBMRegressor__reg_lambda': 0.0, 'LGBMRegressor__silent': True, 'LGBMRegressor__subsam

In [53]:
numeric_models_outliers_scaling_hparams

{'LinearRegression': {'model': Pipeline(steps=[('StandardScaler', StandardScaler()),
                  ('LinearRegression', LinearRegression())]),
  'rmse': 343.3931405442093,
  'params': None},
 'Ridge': {'model': Pipeline(steps=[('StandardScaler', StandardScaler()),
                  ('Ridge', Ridge(alpha=2.0))]),
  'rmse': 338.57558034018706,
  'params': {'memory': None,
   'steps': [('StandardScaler', StandardScaler()),
    ('Ridge', Ridge(alpha=2.0))],
   'verbose': False,
   'StandardScaler': StandardScaler(),
   'Ridge': Ridge(alpha=2.0),
   'StandardScaler__copy': True,
   'StandardScaler__with_mean': True,
   'StandardScaler__with_std': True,
   'Ridge__alpha': 2.0,
   'Ridge__copy_X': True,
   'Ridge__fit_intercept': True,
   'Ridge__max_iter': None,
   'Ridge__normalize': False,
   'Ridge__random_state': None,
   'Ridge__solver': 'auto',
   'Ridge__tol': 0.001}},
 'Lasso': {'model': Pipeline(steps=[('StandardScaler', StandardScaler()),
                  ('Lasso', Lasso(alpha

### VIF Analysis

In [54]:
considered_features = X.columns.tolist()
vif, deleted_features = vif_analysis(X, considered_features, 5)

In [55]:
vif_variables = vif["Variable"].tolist()

In [56]:
vif_models = train_models(X[vif_variables], y, pipelines=pipelines_base_models, cv=5)

In [57]:
vif_models

,model,rmse,regressor
0,LinearRegression,2.980539e-13,(LinearRegression())
1,Ridge,1.610868e-05,(Ridge())
2,Lasso,2.262314e-03,(Lasso())
3,ElasticNet,2.262308e-03,(ElasticNet())
4,KNN,7.467701e+01,(KNeighborsRegressor())
5,CART,4.052014e+01,(DecisionTreeRegressor())
6,RandomForestRegressor,3.213910e+01,(RandomForestRegressor())
7,SVR,4.319876e+02,(SVR())
8,XGBoost,2.136553e+01,"(XGBRegressor(base_score=None, booster=None, c..."
9,LGBMRegressor,1.323498e+02,(LGBMRegressor())


We see that removing multicollinearity from the data set does not improve our results, but decreases them because the variables that are correlated are the ones that are most correlated with the target variable.

### Models PCA

In [58]:
pipelines_pca_models = models_pca_pipelines()
pca_models = train_models(X, y, pipelines=pipelines_pca_models, cv=5)

In [59]:
pca_models

,model,rmse,regressor
0,LinearRegression,4.468937e-12,"(PCA(), LinearRegression())"
1,Ridge,1.895770e-05,"(PCA(), Ridge())"
2,Lasso,1.262661e-01,"(PCA(), Lasso())"
3,ElasticNet,9.608824e-02,"(PCA(), ElasticNet())"
4,KNN,2.164533e+02,"(PCA(), KNeighborsRegressor())"
5,CART,1.831369e+02,"(PCA(), DecisionTreeRegressor())"
6,RandomForestRegressor,1.398131e+02,"(PCA(), RandomForestRegressor())"
7,SVR,4.397186e+02,"(PCA(), SVR())"
8,XGBoost,1.181161e+02,"(PCA(), XGBRegressor(base_score=None, booster=..."
9,LGBMRegressor,1.553723e+02,"(PCA(), LGBMRegressor())"


## Predict Data

In [61]:
models_outliers2

,model,rmse,regressor
0,LinearRegression,302.310170,(LinearRegression())
1,Ridge,302.303977,(Ridge())
2,Lasso,302.011368,(Lasso())
3,ElasticNet,301.500262,(ElasticNet())
4,KNN,304.233294,(KNeighborsRegressor())
5,CART,363.558953,(DecisionTreeRegressor())
6,RandomForestRegressor,248.857616,(RandomForestRegressor())
7,SVR,404.787186,(SVR())
8,XGBoost,266.049053,"(XGBRegressor(base_score=None, booster=None, c..."
9,LGBMRegressor,259.571121,(LGBMRegressor())


In [62]:
predict_model = models_outliers2["regressor"][6]

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X_without_outliers2, y_without_outliers2, test_size=0.33, random_state=42)

In [64]:
 predict_model.fit(X_train, y_train)

Pipeline(steps=[('RandomForestRegressor', RandomForestRegressor())])

In [65]:
predict_model.predict(X_without_outliers2)

array([ 498.26669,  574.81404,  635.86665,   95.465  ,  740.91667,
         89.36   ,   93.395  ,   88.11   , 1010.78959,  773.14167,
        462.24335,  484.71001,  602.86666,  312.23336,  948.2834 ,
         98.55   ,  187.5117 ,  105.755  ,  106.265  ,  647.85833,
       1001.64907,  853.38697,  620.14982,  723.3    ,  844.18127,
       1261.84706,  119.     ,  598.34167,  308.645  ,  735.06665,
         84.535  ,   78.31   ,  183.33668,  275.62502,  211.96   ,
        237.69167,  879.65   ,  885.14999,   80.855  , 1091.375  ,
        701.66666,  336.22   ,  341.79334,  416.80852, 1215.37293,
        120.47167,  344.     ,  231.14668,  219.7767 , 1089.85798,
         82.185  ,  134.24167,  438.40832,  878.61405, 1168.84739,
        589.5417 ,  275.40667,  416.91666,  265.05   ,  556.36665,
       1089.86565,  620.74167, 1082.79491,  118.11167,  435.92499,
        395.55   ,  322.46834,  950.8807 ,   95.35   ,  196.485  ,
        549.14168, 1626.83072,  279.51   ,  668.18332,  332.96